In [339]:
import syft as sy
import numpy as np
from syft.core.adp.entity import DataSubject

## To Do

Download a dataset from Domain

Convert all string columns to unique integers ---> could use hashes


In [340]:
import pandas as pd

In [491]:
canada = pd.read_csv("../../trade_demo/datasets/ca - feb 2021.csv")
italy = pd.read_csv("../../trade_demo/datasets/it - feb 2021.csv")

/home/fizzz/anaconda3/envs/syft/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [492]:
import hashlib
from math import isnan, nan

hash_db = {}
hash_db[nan] = nan

def convert_string(s: str, digits: int=15):
    """Maps a string to a unique hash using SHA, converts it to a hash or an int"""
    if type(s) is str:
        new_hash = int(hashlib.sha256(s.encode("utf-8")).hexdigest(), 16) % 10**digits
        hash_db[s] = new_hash
        return new_hash
    else:
        return s

In [493]:
for col in canada.columns:
    canada[col] = canada[col].map(lambda x: convert_string(x))
for col in italy.columns:
    italy[col] = italy[col].map(lambda x: convert_string(x))

In [346]:
# key_list = list(hash_db.keys())
# val_list = list(hash_db.values())
# position = val_list.index(109781654799833)
# print(key_list[position])

In [494]:
required_columns = ["Classification", "Trade Flow", "Partner", "Commodity Code", "Commodity", "Trade Value (US$)"]

canada_dataset = canada[required_columns]
italy_dataset = italy[required_columns]

In [495]:
canada_dataset.columns

Index(['Classification', 'Trade Flow', 'Partner', 'Commodity Code',
       'Commodity', 'Trade Value (US$)'],
      dtype='object')

In [496]:
italy_dataset.columns

Index(['Classification', 'Trade Flow', 'Partner', 'Commodity Code',
       'Commodity', 'Trade Value (US$)'],
      dtype='object')

In [497]:
canada_dataset = canada_dataset[canada_dataset.Partner == convert_string("Italy")]
italy_dataset = italy_dataset[italy_dataset.Partner == convert_string("Canada")]

In [498]:
result = pd.merge(canada_dataset, italy_dataset, on="Commodity Code")

In [499]:
result.head()

,Classification_x,Trade Flow_x,Partner_x,Commodity Code,Commodity_x,Trade Value (US$)_x,Classification_y,Trade Flow_y,Partner_y,Commodity_y,Trade Value (US$)_y
0,109781654799833,740486968595500,656091199571420,628614662158183,4.036672e+14,8359327,109781654799833,740486968595500,524003986429176,4.036672e+14,2709
1,109781654799833,740486968595500,656091199571420,628614662158183,4.036672e+14,8359327,109781654799833,78789458366718,524003986429176,4.036672e+14,7427990
2,109781654799833,78789458366718,656091199571420,628614662158183,4.036672e+14,26887,109781654799833,740486968595500,524003986429176,4.036672e+14,2709
3,109781654799833,78789458366718,656091199571420,628614662158183,4.036672e+14,26887,109781654799833,78789458366718,524003986429176,4.036672e+14,7427990
4,109781654799833,740486968595500,656091199571420,901412511046831,7.389983e+14,57241,109781654799833,740486968595500,524003986429176,7.389983e+14,6714


In [501]:
np_canada = canada.values
np_italy = italy.values
# np_canada = np_canada.astype(float)
# np_italy = np_italy.astype(float)

In [502]:
required_columns_index = [canada.columns.get_loc(col_name) for col_name in required_columns]

In [503]:
np_canada = np_canada[:, required_columns_index]
np_italy = np_italy[:, required_columns_index]

In [504]:
partner_index = 2

In [505]:
np_canada = np_canada[np.where(np_canada[:, partner_index] == convert_string("Italy"))]
np_italy = np_italy[np.where(np_italy[:, partner_index] == convert_string("Canada"))]

In [506]:
commodity_code_index = 3

In [509]:
bmask1 = [[0] * len(np_canada)] * len(np_italy)

In [510]:
loc = [-1] * len(np_italy)

In [511]:
for i in range(len(np_italy)):
    for j in range(len(np_canada)):
        if np_italy[i][commodity_code_index] == np_canada[j][commodity_code_index]:
            bmask1[i][j] = 1
            loc[i] = j

In [512]:
bmask2 = [0] * len(np_italy)

In [513]:
for i in range(len(np_italy)):
    if loc[i] != -1:
        bmask2[i] = 1
    else:
        bmask2[i] = 0

In [514]:
indices = [i for i, x in enumerate(bmask2) if x]

In [515]:
output = []
for i in indices:
    temp = np.concatenate((np_canada[loc[i]], np_italy[i]))
    output.append(temp)
    
output = np.array(output)

In [516]:
output = np.delete(output, commodity_code_index, 1)